# **Preparacion del ambiente**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#!pip install seaborn
import seaborn as sns
#!pip install plotly==5.7.0
import plotly.express as px
from numpy.ma.core import count

In [6]:
# agregamos código para que identifique más de un display en una celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# **Importación de los datos de properati**


In [12]:
#leo el archivo de properati
#en colab
#csv_path ="/content/drive/MyDrive/DS - Curso/Clases/Desafio 1/Data/properatti.csv"
#data = pd.read_csv(csv_path)

#en jupyter
data = pd.read_csv("../data/properatti.csv", sep = ",", low_memory=False) 
data.shape

(121220, 26)

In [13]:
data.state_name.unique()

array(['Capital Federal', 'Bs.As. G.B.A. Zona Sur',
       'Buenos Aires Costa Atlántica', 'Entre Ríos',
       'Bs.As. G.B.A. Zona Norte', 'Santa Fe', 'Córdoba',
       'Bs.As. G.B.A. Zona Oeste', 'Misiones', 'Buenos Aires Interior',
       'Salta', 'Neuquén', 'Río Negro', 'San Luis', 'Mendoza',
       'Corrientes', 'Chubut', 'Tucumán', 'La Pampa', 'Chaco', 'San Juan',
       'Santa Cruz', 'Tierra Del Fuego', 'Catamarca',
       'Santiago Del Estero', 'Jujuy', 'La Rioja', 'Formosa'],
      dtype=object)

In [14]:
# me quedo solo con AMBA
data = data[(data["state_name"]== 'Capital Federal') | (data["state_name"]== 'Bs.As. G.B.A. Zona Sur') | (data["state_name"]=='Bs.As. G.B.A. Zona Norte') | (data["state_name"]== 'Bs.As. G.B.A. Zona Oeste')]
print(data.shape)

(81150, 26)


In [ ]:
#me quedo solo con amba
mask = data["state_name"] == "Capital Federal"
data_CABA = data[mask]
data_CABA.shape

# **Exploracion y limpieza**


In [15]:
#analizo la columna state_name para ver la distribución y porcentaje de cada valor unique (frecuencia)
percent = (100*data.state_name.value_counts())/data.shape[0]
state_prop = pd.DataFrame({ 'state_name': data.state_name.value_counts(), 'percent': (100*data.state_name.value_counts())/data.shape[0], 'cumpercent': percent.cumsum()})
state_prop

,state_name,percent,cumpercent
Capital Federal,32316,39.822551,39.822551
Bs.As. G.B.A. Zona Norte,25560,31.497227,71.319778
Bs.As. G.B.A. Zona Sur,13952,17.192853,88.512631
Bs.As. G.B.A. Zona Oeste,9322,11.487369,100.000000


In [16]:
#hago un describe de las columnas numéricas sin ninguna limpieza ni imputación para ver valores extremos, y estadísticas básicas
columns = data.select_dtypes('number').columns
data[columns].describe().round()

,Unnamed: 0,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,81150.0,63896.0,46794.0,46794.0,72494.0,72494.0,72494.0,58358.0,72645.0,51635.0,66015.0,5644.0,29939.0,12083.0
mean,59304.0,3481891.0,-35.0,-58.0,365148.0,4686262.0,265593.0,210.0,136.0,2345.0,5282.0,22.0,3.0,5038.0
std,34164.0,388123.0,0.0,0.0,916342.0,6817728.0,386394.0,1111.0,804.0,2875.0,27970.0,135.0,2.0,128803.0
min,0.0,3427208.0,-38.0,-66.0,5000.0,88222.0,5000.0,0.0,0.0,4.0,3.0,1.0,1.0,1.0
25%,29706.0,3429617.0,-35.0,-59.0,110000.0,1852672.0,105000.0,51.0,46.0,1333.0,1600.0,2.0,2.0,1000.0
50%,60366.0,3430992.0,-35.0,-58.0,176000.0,2911342.0,165000.0,87.0,75.0,1980.0,2222.0,3.0,3.0,2000.0
75%,87227.0,3435548.0,-35.0,-58.0,340000.0,5293350.0,300000.0,209.0,159.0,2667.0,3094.0,6.0,4.0,4300.0
max,121219.0,6693230.0,-28.0,-58.0,99999999.0,821271104.0,46545445.0,200000.0,187000.0,206333.0,4000000.0,2509.0,32.0,10001500.0


In [ ]:
#exploramos precios por state y place, para ver posibles valores de imputación
round(data.pivot_table(index='state_name', columns='property_type',
                    aggfunc={ 'price_usd_per_m2':'median'}),2)

In [ ]:
#exploramos precios por state y place, para ver posibles valores de imputación
round(data.pivot_table(index=['state_name','place_name'], columns='property_type',
                    aggfunc={ 'price_usd_per_m2':'median'}),2)

## Elimino columnas que no use

In [17]:
cols2keep = ['property_type', 'state_name', 'place_name', 'place_with_parent_names','price_aprox_usd', 'surface_total_in_m2','surface_covered_in_m2',
             'price_usd_per_m2', 'rooms', 'description', 'title', 'properati_url']
data = data.loc[:, cols2keep]
#print(df_sub)

In [18]:
##  ¿Qué campos tienen valores nulos? y cual es su porcentaje en la columna
cant_nulos_por_campo = data.apply(lambda x: x.isnull().sum(), axis=0)
percent_nulos_por_campo = data.apply(lambda x: (100 * x.isnull().sum() / data.shape[0]).round(2), axis=0)
summary_nulos_por_campo = pd.DataFrame({ 'cant': cant_nulos_por_campo, 'percent': percent_nulos_por_campo ,'tipo': data.dtypes})
print(summary_nulos_por_campo)

                          cant  percent     tipo
property_type                0     0.00   object
state_name                   0     0.00   object
place_name                  23     0.03   object
place_with_parent_names      0     0.00   object
price_aprox_usd           8656    10.67  float64
surface_total_in_m2      22792    28.09  float64
surface_covered_in_m2     8505    10.48  float64
price_usd_per_m2         29515    36.37  float64
rooms                    51211    63.11  float64
description                  1     0.00   object
title                        0     0.00   object
properati_url                0     0.00   object


## Elimino registros duplicados

In [19]:
## elimino duplicados 
data.drop_duplicates(keep="first")

,property_type,state_name,place_name,place_with_parent_names,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,rooms,description,title,properati_url
0,PH,Capital Federal,Mataderos,|Argentina|Capital Federal|Mataderos|,62000.0,55.0,40.0,1127.272727,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,http://www.properati.com.ar/15bo8_venta_ph_mat...
1,apartment,Bs.As. G.B.A. Zona Sur,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,150000.0,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,http://www.properati.com.ar/15bob_venta_depart...
2,apartment,Capital Federal,Mataderos,|Argentina|Capital Federal|Mataderos|,72000.0,55.0,55.0,1309.090909,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,http://www.properati.com.ar/15bod_venta_depart...
3,PH,Capital Federal,Liniers,|Argentina|Capital Federal|Liniers|,95000.0,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,http://www.properati.com.ar/15boh_venta_ph_lin...
6,PH,Bs.As. G.B.A. Zona Norte,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,130000.0,106.0,78.0,1226.415094,NaN,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",http://www.properati.com.ar/15bor_venta_ph_mun...
...,...,...,...,...,...,...,...,...,...,...,...,...
121214,store,Bs.As. G.B.A. Zona Norte,San Isidro,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|,NaN,123.0,123.0,NaN,NaN,***VENTA CON RENTA***Local en EDIFICIO lomas ...,Local - San Isidro,http://www.properati.com.ar/1cja1_venta_local_...
121215,apartment,Capital Federal,Belgrano,|Argentina|Capital Federal|Belgrano|,870000.0,113.0,93.0,7699.115044,NaN,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,http://www.properati.com.ar/1cja2_venta_depart...
121216,house,Bs.As. G.B.A. Zona Norte,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,498000.0,360.0,360.0,1383.333333,NaN,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,http://www.properati.com.ar/1cja6_venta_casa_b...
121217,apartment,Capital Federal,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,131500.0,46.0,39.0,2858.695652,NaN,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,http://www.properati.com.ar/1cja7_venta_depart...


## paso todo a minúscula

In [20]:
## pasar todas las columnas a minusculas
data_lower = data.applymap(lambda x: x if np.isreal(x) else str(x).lower())
# comparo los tipos de datos antes y después de pasar a minúsculas:
print(data_lower.dtypes == data.dtypes)

property_type              True
state_name                 True
place_name                 True
place_with_parent_names    True
price_aprox_usd            True
surface_total_in_m2        True
surface_covered_in_m2      True
price_usd_per_m2           True
rooms                      True
description                True
title                      True
properati_url              True
dtype: bool


## Elimino outliers en price_usd_per_m2

In [23]:
#búsqueda y reemplazo de outliers (de más de 2 std, 95%) por NaN en las columnas numéricas, en un solo paso
df_sub = data.loc[:, 'price_usd_per_m2']
lim = np.abs((df_sub - df_sub.mean()) / df_sub.std(ddof=0)) < 1.75
data.loc[:, 'price_usd_per_m2'] = df_sub.where(lim, np.nan)
#data.head(3)
data.shape
#data.dropna()

(81150, 12)

In [24]:
data.price_usd_per_m2.describe()

count    48008.000000
mean      1907.482369
std        852.905744
min        109.090909
25%       1298.652840
50%       1904.761905
75%       2500.000000
max       4006.711409
Name: price_usd_per_m2, dtype: float64

# **Imputaciones**


## Habitaciones


In [25]:
totalVentas = data.state_name.notnull().sum()
totalSinCuartos = data.rooms.isnull().sum()
porcentajeSinCuartos = round( (totalVentas- totalSinCuartos) / totalVentas * 100, 2)
print('Total Registros \t\t\t', totalVentas )
print('Total sin ambientes \t\t\t', totalSinCuartos )
print('Porcentaje sin ambientes \t\t', porcentajeSinCuartos, ' %' )


Total Registros 			 81150
Total sin ambientes 			 51211
Porcentaje sin ambientes 		 36.89  %


El dataset esta completo en un 39.09% para todas las propiedades, podemos hacer foco en los departamentos:


In [26]:
data['title'] = data.title.str.upper()
data['description'] = data.description.str.upper()

Reemplazamos los números escritos como palabras y todas la variantes de llamar a los monoambientes por '1 AMBIENTE'.

In [27]:
reemplazo_dic = {"UNO":"1", "DOS":"2", "TRES":"3", "CUATRO":"4", "CINCO":"5", "SEIS":"6", "SIETE":"7", "OCHO":"8",
             "NUEVE":"9", "DIEZ":"10", "MONOAMBIENTE":"1 AMBIENTE", "MONOAMB" : "1 AMBIENTE", "UN":"1", "AMBIENTE DIVISIBLE":"1 AMBIENTE",
             "MONO AMBIENTE": "1 AMBIENTE","DORMITORIOS": "AMBIENTE","DORMITORIO": "AMBIENTE","HABITACIONES": "AMBIENTE","HABITACION": "AMBIENTE"}
for key in reemplazo_dic.keys():
    data.description = data.description.str.replace(key, reemplazo_dic[key], regex=False)

for key in reemplazo_dic.keys():
    data.title = data.title.str.replace(key, reemplazo_dic[key], regex=False)

Recuperamos los ambientes desde título, descripción y actualizamos rooms.

patrones_amb = ["(\d+) AMB","(\d+)AMB"]
patrones_dorm = ["(\d+) DORM","(\d+)DORM","(\d+)HABITACIO","(\d+) HABITACIO"]
cols = (controlRooms.description, controlRooms.title)

for patron in patrones_dorm:
    for col in cols:
        col.str.extract(patron).astype(float) + 1
        data.update(controlRooms)

for patron in patrones_amb:
    for col in cols:
        col.str.extract(patron).astype(float)
        data.update(controlRooms)       

In [28]:
controlRooms = data[(data.rooms.isnull())]

In [29]:
controlRooms.rooms = controlRooms["description"].str.extract("(\d+) AMB").astype(float)
data.update(controlRooms)

C:\Users\maela\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [30]:
controlRooms.rooms = controlRooms.title.str.extract("(\d+) AMB").astype(float)
data.update(controlRooms)

In [31]:
controlRooms.rooms = controlRooms.description.str.extract("(\d+) DORM").astype(float) + 1
data.update(controlRooms)

In [32]:
controlRooms.rooms = controlRooms.title.str.extract("(\d+) DORM").astype(float) + 1
data.update(controlRooms)

In [33]:
controlRooms.rooms = controlRooms.title.str.extract("(\d+)AMB").astype(float)
data.update(controlRooms)

In [34]:
controlRooms.rooms = controlRooms.description.str.extract("(\d+)AMB").astype(float)
data.update(controlRooms)

In [35]:
controlRooms.rooms = controlRooms.title.str.extract("(\d+)DORM").astype(float) + 1
data.update(controlRooms)

In [36]:
controlRooms.rooms = controlRooms.description.str.extract("(\d+) DORM").astype(float) + 1
data.update(controlRooms)

In [37]:
controlRooms.rooms = controlRooms.title.str.extract("(\d+)HABITACIO").astype(float) + 1
data.update(controlRooms)

In [38]:
controlRooms.rooms = controlRooms.title.str.extract("(\d+) HABITACIO").astype(float) + 1
data.update(controlRooms)

In [39]:
controlRooms.rooms = controlRooms.description.str.extract("(\d+)HABITACIO").astype(float) + 1
data.update(controlRooms)

In [40]:
controlRooms.rooms = controlRooms.description.str.extract("(\d+) HABITACIO").astype(float) + 1
data.update(controlRooms)

In [41]:
controlRooms = data[(data.rooms.isnull())]

Volvemos a controlar luego del proceso el grado de llenado de la columna

In [42]:
totalVentas = data.state_name.notnull().sum()
totalSinCuartos = data.rooms.isnull().sum()
porcentajeSinCuartos = round( (totalVentas- totalSinCuartos) / totalVentas * 100, 2)
print('Total Registros \t\t\t', totalVentas )
print('Total sin Ambientes \t\t\t', totalSinCuartos )
print('Porcentaje con ambientes \t\t', porcentajeSinCuartos, ' %' )

Total Registros 			 81150
Total sin Ambientes 			 10692
Porcentaje con ambientes 		 86.82  %


Luego del proceso se paso del 39,09 % de llenado para todas las propiedades del data set al 86,7 %. Veamos que pasa para cada tipo de propiedad

## chequeo de nuevo valores nulls

In [43]:
##  ¿Qué campos tienen valores nulos? y cual es su porcentaje en la columna
cant_nulos_por_campo = data.apply(lambda x: x.isnull().sum(), axis=0)
percent_nulos_por_campo = data.apply(lambda x: (100 * x.isnull().sum() / data.shape[0]).round(2), axis=0)
summary_nulos_por_campo = pd.DataFrame({ 'cant': cant_nulos_por_campo, 'percent': percent_nulos_por_campo ,'tipo': data.dtypes})
print(summary_nulos_por_campo)

                          cant  percent     tipo
property_type                0     0.00   object
state_name                   0     0.00   object
place_name                  23     0.03   object
place_with_parent_names      0     0.00   object
price_aprox_usd           8656    10.67  float64
surface_total_in_m2      22792    28.09  float64
surface_covered_in_m2     8505    10.48  float64
price_usd_per_m2         33142    40.84  float64
rooms                    10692    13.18  float64
description                  1     0.00   object
title                        0     0.00   object
properati_url                0     0.00   object


## Place_with_parent_names


In [ ]:
# Creo tantas variables como niveles tiene la columna place_with_parent_names procurando no meter valores vacíos tipo '' o ' '. 
#En cada columna pongo nan (np.NaN) si no hay valor en ese nivel.
pais = data.place_with_parent_names.apply(lambda x: x.split('|')[1] if (len(x.split('|')) >= 2 and x.split('|')[1] != '') else np.NaN)
provincia = data.place_with_parent_names.apply(lambda x: x.split('|')[2] if (len(x.split('|')) >= 3 and x.split('|')[2] != '') else np.NaN)
localidad = data.place_with_parent_names.apply(lambda x: x.split('|')[3] if (len(x.split('|')) >= 4 and x.split('|')[3] != '') else np.NaN)
barrio = data.place_with_parent_names.apply(lambda x: x.split('|')[4] if (len(x.split('|')) >= 5 and x.split('|')[4] != '') else np.NaN)
barrio_1 = data.place_with_parent_names.apply(lambda x: x.split('|')[5] if (len(x.split('|')) >= 6 and x.split('|')[5] != '') else np.NaN)

# Chequeo si las columnas nativas son iguales a las creadas de nuevo, al menos las superiores
(data.country_name != pais).sum()
(data.state_name != provincia).sum()

# Para las columnas que no son iguales o no tienen contraparte (place_name tiene la granularidad más fina por observación, con una mezcla de provincias, localidades y barrios) 
#hago la suma de sus componentes para ver si completamos el total de observaciones (121220).
provincia.loc[localidad.isnull()].notnull().sum() + localidad.loc[barrio.isnull()].notnull().sum() + barrio.loc[barrio_1.isnull()].notnull().sum() + barrio_1.notnull().sum() - len(data)

# Chequeo que place_name tampoco tenga nulos.
data.place_name.isnull().sum()

# Como sí tiene, miro cuáles son y con qué debería estar completo
data.loc[data.place_name.isnull()].head(3)

In [ ]:
# Agrego las columnas de lugares desagregados al dataframe original
data['s_pais'], data['s_provincia'], data['s_localidad'], data['s_barrio'], data['s_barrio_1'] = pais, provincia, localidad, barrio, barrio_1

In [ ]:
# Chequeo que todo lo que hay en place_name esté en las nuevas columnas accesorias
data.place_name.loc[data.apply(lambda x: np.NaN if x['place_name'] == x['s_barrio_1'] else (np.NaN if x['place_name'] == x['s_barrio'] else (np.NaN if x['place_name'] == x['s_localidad'] else (np.NaN if x['place_name'] == x['s_provincia'] else x['place_name']))), axis = 1).notnull()]

In [ ]:
# Creo una nueva columna que tenga lo más fino de los lugares por observación, en parte para imputar los nans de place_name, y en parte para corroborar que place_name tenga en realidad lo más fino y no algún nivel superior
data['s_place_name'] = data.apply(lambda x: x['s_barrio_1'] if x['s_barrio_1'] is not np.NaN else (x['s_barrio'] if x['s_barrio'] is not np.NaN else (x['s_localidad'] if x['s_localidad'] is not np.NaN else (x['s_provincia'] if x['s_provincia'] is not np.NaN else x['pais']))), axis = 1)

In [ ]:
# Chequeo si la columna recién creada tiene algú null
data['s_place_name'].isnull().sum()
# Ahora chequeo cuántas diferencias tiene con place_name
(data['s_place_name'] != data['place_name']).sum()
# Con esto vemos que sólo diferían en los nulos y que ahora fueron completados. Veamos si los completamos bien (deberían decir 'Tigre')
data[['place_name', 'place_with_parent_names', 's_place_name']].loc[data.place_name.isnull()]

In [ ]:
# Creo los dataframes agrupados por lugar y tipo de propiedad para luego hacer los merges
data_grp_b1 = data.groupby(['s_barrio_1', 'property_type'])['price_usd_per_m2'].median().reset_index()
data_grp_b = data.groupby(['s_barrio', 'property_type'])['price_usd_per_m2'].median().reset_index()
data_grp_l = data.groupby(['s_localidad', 'property_type'])['price_usd_per_m2'].median().reset_index()
data_grp_p = data.groupby(['s_provincia', 'property_type'])['price_usd_per_m2'].median().reset_index()

In [ ]:
# mergeo el dataset original con los dataframes agrupados creando cada vez una columna nueva con las medianas por lugar desde lo más fino (barrio_1 o b1) hasta lo más grueso (provincia o p)

# merge barrio_1
data_merge = pd.merge(data, data_grp_b1, left_on = ["s_barrio_1","property_type"], right_on = ["s_barrio_1","property_type"], how = "left", suffixes = ('', '_b1'))

# merge barrio 
data_merge = pd.merge(data_merge, data_grp_b, left_on = ["s_barrio","property_type"], right_on = ["s_barrio","property_type"], how = "left", suffixes = ('', '_b'))

# merge localidad 
data_merge = pd.merge(data_merge, data_grp_l, left_on = ["s_localidad","property_type"], right_on = ["s_localidad","property_type"], how = "left", suffixes = ('', '_l'))

# merge provincia 
data_merge = pd.merge(data_merge, data_grp_p, left_on = ["s_provincia","property_type"], right_on = ["s_provincia","property_type"], how = "left", suffixes = ('', '_p'))

data_merge.head(3)

In [ ]:
# Ahora creo la nueva columna de inputación de precios en dólares por m2 poniendo el valor original si existe (price_usd_per_m2) o la inputación más fina que tengamos (barrio_1 si no es nan, si no barrio si no es nan, si no localidad si no es nan, y si no provincia)
data_merge['input_price_usd_m2'] = data_merge.apply(lambda x: x['price_usd_per_m2'] if pd.notna(x['price_usd_per_m2']) else (x['price_usd_per_m2_b1'] if pd.notna(x['price_usd_per_m2_b1']) else (x['price_usd_per_m2_b'] if pd.notna(x['price_usd_per_m2_b']) else (x['price_usd_per_m2_l'] if pd.notna(x['price_usd_per_m2_l']) else x['price_usd_per_m2_p']))), axis = 1)
data_merge.head(3)

In [ ]:
# Dropeo las columnas accesorias que no suman info al dataset general
data_merge.drop(labels = ['s_pais', 's_provincia', 's_localidad', 's_barrio', 's_barrio_1', 's_place_name', 'price_usd_per_m2_b1', 'price_usd_per_m2_b', 'price_usd_per_m2_l', 'price_usd_per_m2_p'], axis = 1, inplace = True)
data_merge.head(3)

In [ ]:
# Porcentaje de nan en los precios por m2 en dólares imputados
data_merge.input_price_usd_m2.isnull().sum() / len(data_merge) * 100

# Describe
data_merge.input_price_usd_m2.describe()

# Coeficiente de variación
data_merge.input_price_usd_m2.std() / data_merge.input_price_usd_m2.mean()

## Elimino registros con valores NaN

In [45]:
data.dropna(axis=0, how='any', subset=['property_type', 'state_name', 'place_name', 'price_aprox_usd', 'surface_total_in_m2','surface_covered_in_m2', 'rooms', 'price_usd_per_m2'], inplace=True)

In [46]:
data.shape

(40561, 12)

In [47]:
##  ¿Qué campos tienen valores nulos? y cual es su porcentaje en la columna
cant_nulos_por_campo = data.apply(lambda x: x.isnull().sum(), axis=0)
percent_nulos_por_campo = data.apply(lambda x: (100 * x.isnull().sum() / data.shape[0]).round(2), axis=0)
summary_nulos_por_campo = pd.DataFrame({ 'cant': cant_nulos_por_campo, 'percent': percent_nulos_por_campo ,'tipo': data.dtypes})
print(summary_nulos_por_campo)

                         cant  percent     tipo
property_type               0      0.0   object
state_name                  0      0.0   object
place_name                  0      0.0   object
place_with_parent_names     0      0.0   object
price_aprox_usd             0      0.0  float64
surface_total_in_m2         0      0.0  float64
surface_covered_in_m2       0      0.0  float64
price_usd_per_m2            0      0.0  float64
rooms                       0      0.0  float64
description                 0      0.0   object
title                       0      0.0   object
properati_url               0      0.0   object


# Creacion columnas dummies

## Columnas dummies amenities



In [ ]:
#fracciono la columna properti_url para sacar la nube de palabras mas repetidas
import re
patron_url = re.compile(pattern="_", flags =re.IGNORECASE)
lista_url = data["properati_url"].apply(lambda x : patron_url.split(x))
serie_palabras = pd.Series(np.hstack(lista_url))
#serie_palabras.value_counts().head(20).plot(kind="bar")

In [ ]:
serie_palabras.value_counts().head(50)

venta                            121086
departamento                      71019
garage                            56620
lavadero                          44034
balcon                            42286
casa                              40196
parrilla                          33381
piscina                           32146
luminoso                          31865
suite                             27101
placard                           26145
terraza                           25101
toilette                          22841
patio                             21168
vestidor                          18011
jardin                            17131
quincho                           12848
aire-acondicionado                12763
sum                               11665
dependencias                      11316
amenities                         10607
baulera                           10512
estrenar                          10379
vista                             10011
gimnasio                           9363


In [ ]:
# a partir de la nube de palabras selecciono las que son buenos adicionales
adicionales = ["garage", "balcon", "parrilla", "piscina", "terraza", "patio", "jardin", "quincho", "sum", "amenities", "baulera", "gimnasio", "subte-linea-d", "subte-linea-b", "subte-linea-a", "subte-linea-h", "subte-linea-e" ]


In [ ]:
#elimino el primer elemento de lista_url para no tener el elemento con el http: etc
for sublist in lista_url:
  del sublist[0]

In [ ]:
#lista_url

0         [venta, ph, mataderos, lavadero, patio, inmobi...
1         [venta, departamentos, la-plata, balcon, lavad...
2         [venta, departamentos, mataderos, lavadero, pl...
3                   [venta, ph, liniers, patio, g-goffredo]
4         [venta, departamentos, centro, cristina-pavone...
                                ...                        
121215    [venta, departamento, belgrano, balcon, suite,...
121216    [venta, casa, beccar, suite, hidromasaje, jard...
121217    [venta, departamento, villa-urquiza, holmberg,...
121218    [venta, departamento, plaza-colon, lavadero, l...
121219    [venta, departamento, capital-federal, baulera...
Name: properati_url, Length: 121220, dtype: object

In [ ]:
#creo una función que compare la lista de palabras con la lista de listas
#y me da como resultado una lista de listas de palabras true/false segun coincida o no 
def buscador_palabras(quebuscar, dondebuscar):
  listadeextras = []
  for listas in dondebuscar:
    extras = []
    for palabra in quebuscar:
      if palabra in listas:
        extras.append(True)
      else:
        extras.append(False)
    listadeextras.append(extras)
  #print(listadeextras)
  return listadeextras     

In [ ]:
#aplico la funcion a mi lista "adicionales" y "lista_url"
#chequeo que tenga la misma longitud de data
resultado = buscador_palabras(adicionales,lista_url)
len(resultado)

121220

In [ ]:
#convierto resultado en dataframe, y renombro las columnas por la lista de palabras adicionales
df = pd.DataFrame(resultado)
df.columns = ["garage", "balcon", "parrilla", "piscina", "terraza", "patio", "jardin", "quincho", "s.u.m.", "amenities", "baulera", "gimnasio","subte-linea-d", "subte-linea-b", "subte-linea-a", "subte-linea-h", "subte-linea-e"]
df

,garage,balcon,parrilla,piscina,terraza,patio,jardin,quincho,s.u.m.,amenities,baulera,gimnasio
0,False,False,False,False,False,True,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
121215,True,True,True,True,False,False,False,True,False,False,False,False
121216,True,False,True,True,False,False,True,True,False,False,False,False
121217,True,True,True,False,True,False,False,False,False,True,False,False
121218,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
#uno el dataframe original con el nuevo generado de true/false
data = pd.merge(data,df,left_index=True, right_index=True)
data.columns
#antes me puso los dos indices como resultado del merge, y tuve que sacar la primera columna
#data.drop(columns=data.columns[0], axis=1,inplace=True)
#data.columns

In [ ]:
columnas = [data['garage'], data['balcon'], data['parrilla'], data['piscina'],
       data['terraza'], data['patio'], data['jardin'], data['quincho'], data['s.u.m.'], data['amenities'], data['baulera'],
       data['gimnasio']]
for columna in columnas:
  print(columna.name, columna.sum())

## Columnas dummies State_Name y Place_name

# Exploracion post creacion de variables

In [ ]:
#EXPLORACION PRECIOS EN DOLARES POR M2, POR PROVINCIA
round(data.pivot_table(index='state_name', columns='property_type',
                    aggfunc={ 'price_usd_per_m2':'median'}),2)

In [44]:
##  ¿Qué campos tienen valores nulos? y cual es su porcentaje en la columna
cant_nulos_por_campo = data.apply(lambda x: x.isnull().sum(), axis=0)
percent_nulos_por_campo = data.apply(lambda x: (100 * x.isnull().sum() / data.shape[0]).round(2), axis=0)
summary_nulos_por_campo = pd.DataFrame({ 'cant': cant_nulos_por_campo, 'percent': percent_nulos_por_campo ,'tipo': data.dtypes})
print(summary_nulos_por_campo)

                          cant  percent     tipo
property_type                0     0.00   object
state_name                   0     0.00   object
place_name                  23     0.02   object
place_with_parent_names      0     0.00   object
price_aprox_usd          20410    16.84  float64
surface_total_in_m2      39328    32.44  float64
surface_covered_in_m2    19907    16.42  float64
price_usd_per_m2         67406    55.61  float64
price_per_m2             33562    27.69  float64
rooms                    16055    13.24  float64
description                  2     0.00   object
title                        0     0.00   object
properati_url                0     0.00   object


- columna que complete con la media del precio por m2 según place_name

In [ ]:
#creo una columna que rellene el valor con la media del precio por m2 en este state_name
data_merge["price_state"] = data_merge.groupby('state_name')['price_usd_per_m2'].transform('median') 
data_merge.head()

# **Exportación de resultados**

In [ ]:
#creo un nuevo dataset de salida con los métodos aplicados
# exportar a colab
#df.to_csv(r'/content/drive/MyDrive/DSDH/TP1/properatti_nuevo.csv', index = False, header=True)
#exportar local
#data.to_csv(r'properatti_nuevo.csv', index = False, header=True)

# **Machine Learning**

## Sklearn

In [ ]:
# Definimos parámetros globales para matplotlib.
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 16

In [ ]:
# importamos el modelo lineal y algunas funciones para calcular la bondad de ajuste.
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# Seleccionamos la variable predictora y la objetivo.
X = df[[features]]
y = targets[target]

# Importamos, Instanciamos, Fiteamos, etc..

# Instanciamos el modelo.
lm = linear_model.LinearRegression()

# Fiteamos el modelo sobre los vectores X e y.
model = lm.fit(X, y)
#
# Guardamos  las predicciones en un nuevo vector que llamaremos predictions.
predictions = lm.predict(X)

# Imprimimos el intercepto y los coeficientes como atributos del objeto entrenado.
print ('Intercepto=', ' ', model.intercept_)
print ('RM=', ' ', model.coef_)
# imprimos la metrica que mide la bondad de ajusto del modelo. En este caso el R2.
print ('R2_train=', ' ', model.score(X, y))

In [ ]:
# Generamos una función que resume los coeficientes, el intercepto y el R2
# "model" = objeto con el modelo
# "X" = matrix de variables independientes

def sum_mod(model, X):
    a = pd.DataFrame(model.coef_ , X.columns.values)
    a = a.append(pd.DataFrame([model.intercept_, model.score(X, y)], index=['Intecept','R2']))
    return(a)

In [ ]:
# Graficamos el modelo
plt.plot(y,y, '-.',c='grey')
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicciones usando (una, dos, todas) las variables")
plt.ylabel("Valores reales ")
plt.show()
print ("EMC:", mean_squared_error(y, predictions))
print ("¿Mejora?: ", mean_squared_error(y, predictions) < prevMSE)

## Statsmodel

In [ ]:
# Importamos la api.
import statsmodels.api as sm

# De manera análoga a la vista en el primer ejercicio, definimos el vector de variables con la primer variable RM.
X = df[[features]]
y = df[[target]]

# Tenemos que agregar explícitamente a una constante:
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

# Graficamos los resultados
plt.plot(y,y, '-.', c='grey')
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicciones")
plt.ylabel("Valores reales target")
plt.show()

# Imprimimos el MSE y un resumen del modelo
print ("EMC:", mean_squared_error(y, predictions))
print (model.summary())

In [ ]:
#para CLMultiple
# visualizamos la matriz de correlación en Seaborn usando a heatmap

sns.heatmap(bikes.corr(), vmin=-1, vmax=1, center=0, cmap="YlGnBu");

In [ ]:
#elegir entre modelos
from sklearn import metrics
import numpy as np
print ('MAE:', metrics.mean_absolute_error(true, pred))
print ('MSE:', metrics.mean_squared_error(true, pred))
print ('RMSE:', np.sqrt(metrics.mean_squared_error(true, pred)))
print ('R2:', metrics.r2_score(true, pred))

In [ ]:
# Definimos una función que acepta una lista de features, hace el split entre train y test,
# reservando un 25% de las observaciones para testeo, y devuelve la prueba RMSE.

from sklearn.model_selection import train_test_split

def train_test_rmse(feature_cols):
    X = bikes[feature_cols]
    y = bikes.total
    # Como estamos trabajando con observaciones ordenadas en el tiempo, ponemos
    # shuffle=False para evitar data leakage
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    return np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [ ]:
# Chequeamos que las columnas son perfectamente dependientes.
np.all(bikes.casual + bikes.registered == bikes.total)

## Compruebo los supuestos Gauss Markov

In [ ]:
#linearidad del modelo
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

def linearity_test(model, y):
    '''
    funcion para visualizar e identificar supuestos de linealidad sobre la regression lineal
    
    Args:
    * model - fitted OLS model from statsmodels
    * y - observed values
    '''
    fitted_vals = model.predict()
    resids = model.resid

    fig, ax = plt.subplots(1,2)
    
    sns.regplot(x=fitted_vals, y=y, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Observados vs. Valores Predichos', fontsize=16)
    ax[0].set(xlabel='Predichos', ylabel='Observados')

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Residos vs. Valores Predichos', fontsize=16)
    ax[1].set(xlabel='Predichos', ylabel='Residuos')
    
linearity_test(lin_reg, y)  

In [ ]:
#media de los residuos
lin_reg.resid.mean()

In [ ]:
#multicolinearidad uso IVF(inflación de varianza) muestra cuánto más grande es el error estándar, en comparación 
#con lo que sería si ese predictor no estuviera correlacionado con las otras características del modelo . 
#Si no se correlacionan características, todos los valores para VIF serán 1.
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor(X_constant.values, i) for i in range(X_constant.shape[1])]
pd.DataFrame({'vif': vif[1:]}, index=X.columns).T

In [ ]:
#homocedasticidad
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

def homoscedasticity_test(model):
    '''
    Function for testing the homoscedasticity of residuals in a linear regression model.
    It plots residuals and standardized residuals vs. fitted values and runs Breusch-Pagan and Goldfeld-Quandt tests.
    
    Args:
    * model - fitted OLS model from statsmodels
    '''
    import numpy as np
    fitted_vals = model.predict()
    resids = model.resid
    resids_standardized = model.get_influence().resid_studentized_internal

    fig, ax = plt.subplots(1,2)

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Residuals vs Fitted', fontsize=16)
    ax[0].set(xlabel='Fitted Values', ylabel='Residuals')

    sns.regplot(x=fitted_vals, y=np.sqrt(np.abs(resids_standardized)), lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Scale-Location', fontsize=16)
    ax[1].set(xlabel='Fitted Values', ylabel='sqrt(abs(Residuals))')

    bp_test = pd.DataFrame(sms.het_breuschpagan(resids, model.model.exog), 
                           columns=['value'],
                           index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'])

    gq_test = pd.DataFrame(sms.het_goldfeldquandt(resids, model.model.exog)[:-1],
                           columns=['value'],
                           index=['F statistic', 'p-value'])

    print('\n Breusch-Pagan test ----')
    print(bp_test)
    print('\n Goldfeld-Quandt test ----')
    print(gq_test)
    print('\n Residuals plots ----')

homoscedasticity_test(lin_reg)

In [ ]:
#autocorrelacion
import statsmodels.tsa.api as smt

acf = smt.graphics.plot_acf(lin_reg.resid, lags=40 , alpha=0.05)
#acf.show();

In [ ]:
#normalidad de los residuos
from scipy import stats

def normality_of_residuals_test(model):
    '''
    Function for drawing the normal QQ-plot of the residuals and running 4 statistical tests to 
    investigate the normality of residuals.
    
    Arg:
    * model - fitted OLS models from statsmodels
    '''
    sm.ProbPlot(model.resid).qqplot(line='s');
    plt.title('Q-Q plot');

    jb = stats.jarque_bera(model.resid)
    sw = stats.shapiro(model.resid)
    ad = stats.anderson(model.resid, dist='norm')
    ks = stats.kstest(model.resid, 'norm')
    
    print(f'Jarque-Bera test ---- statistic: {jb[0]:.4f}, p-value: {jb[1]}')
    print(f'Shapiro-Wilk test ---- statistic: {sw[0]:.4f}, p-value: {sw[1]:.4f}')
    print(f'Kolmogorov-Smirnov test ---- statistic: {ks.statistic:.4f}, p-value: {ks.pvalue:.4f}')
    print(f'Anderson-Darling test ---- statistic: {ad.statistic:.4f}, 5% critical value: {ad.critical_values[2]:.4f}')
    print('If the returned AD statistic is larger than the critical value, then for the 5% significance level, the null hypothesis that the data come from the Normal distribution should be rejected. ')
    
normality_of_residuals_test(lin_reg)

## Variables dummies

In [ ]:
#pandas (drop_first para evitar la colinearidad perfecta)
dummy = pd.get_dummies(df[column], drop_first=True)

In [ ]:
#scikit onehotencoder
from sklearn.preprocessing import OneHotEncoder 
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
# fiteo y transformo la columna "sex"
dummy_oneHot = onehot_encoder.fit_transform(df[['Sex']])
# pongo un vector en un dataset.
dummy_oneHot = pd.DataFrame(dummy_oneHot.toarray(),columns=df['Sex'].unique())
dummy_oneHot.head()

#con drop_first
from sklearn.preprocessing import OneHotEncoder 
onehot_encoder2 = OneHotEncoder(drop='first')
# fiteo y transformo la columna "sex"
dummy_oneHot_correct = onehot_encoder2.fit_transform(df[['Sex']])
# pongo un vector en un dataset.
dummy_oneHot_correct = pd.DataFrame(dummy_oneHot_correct.toarray())
dummy_oneHot_correct.head()

In [ ]:
#para variables categoricas
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(df['Survived'])

## Normalización

In [ ]:
#metodo manual con mean y std
xs = df["NOX"]
ys = df["TAX"]
plt.scatter(xs, ys, color='b')
plt.xlabel("NOX")
plt.ylabel("TAX")
plt.show()

xs = df["NOX"]
mean = np.mean(xs)
std = np.std(xs)
xs = [(x - mean) / std for x in xs]

ys = df["TAX"]
mean = np.mean(ys)
std = np.std(ys)
ys = [(y - mean) / std for y in ys]

plt.scatter(xs, ys, color='r')
plt.xlabel("NOX standardized")
plt.ylabel("TAX standardized")
plt.show()

In [ ]:
from sklearn import preprocessing

xs = preprocessing.scale(df["NOX"])
ys = preprocessing.scale(df["TAX"])

plt.scatter(xs, ys, color='r')
plt.xlabel("NOX standardized")
plt.ylabel("TAX standardized")
plt.show()



In [ ]:
#manual min max
xs = df["NOX"]
ys = df["TAX"]
plt.scatter(xs, ys, color='b')
plt.xlabel("NOX")
plt.ylabel("TAX")
plt.show()

xs = df["NOX"]
xmin = np.min(xs)
xmax = np.max(xs)
xs = [(x - xmin) / (xmax - xmin) for x in xs]

ys = df["TAX"]
ymin = np.min(ys)
ymax = np.max(ys)
ys = [(y - ymin) / (ymax - ymin) for y in ys]

plt.scatter(xs, ys, color='r')
plt.xlabel("NOX Min-Max Scaled")
plt.ylabel("TAX Min-Max Scaled")
plt.show()

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()

xs = scaler.fit_transform(df[["NOX"]])
ys = scaler.fit_transform(df[["TAX"]])

plt.scatter(xs, ys, color='r')
plt.xlabel("NOX Min-Max Scaled")
plt.ylabel("TAX Min-Max Scaled")
plt.show()

#o
numericals = ['carat', 'depth', 'table', 'x', 'y', 'z']

X = data[numericals]

scaler = MinMaxScaler()
scaler.fit(X)

std_numerical_data = scaler.transform(X)
std_df = pd.DataFrame(std_numerical_data)
std_df.columns = [i + '_std' for i in numericals]
std_df


## Regularización

### Ridge Regression

In [ ]:
rlm = linear_model.Ridge(alpha=0.5, normalize=True)

# Ajustamos nuevamente, esta vez con regularizacion
X = np.vander(xs, 4)[:,:-1]
y = ys

ridge_model = rlm.fit(X, y)
predictions = ridge_model.predict(X)

plt.scatter(xs, ys)

plt.title("Muestra de datos #1")
plt.scatter(xs, predictions, c='r')
print ("r^2:", ridge_model.score(X, ys))

In [ ]:
X = np.vander(xs2, 4)[:,:-1]
predictions = ridge_model.predict(X)

plt.scatter(xs2, ys2)
plt.title("Muestra de datos#2")
plt.scatter(xs2, predictions, c='r')

print ("r^2:", ridge_model.score(X, ys2))

In [ ]:
# Veamos los coeficientes de la regresión Ridge:    
ridge_model.coef_

### Regression Lasso

In [ ]:
lasso = linear_model.Lasso(alpha=0.5, normalize=True)

# Ajustamos nuevamente, esta vez con regularizacion
X = np.vander(xs, 4)[:,:-1]
y = ys
lasso_model =lasso.fit(X, y)
predictions = lasso_model.predict(X)

plt.scatter(xs, ys)
plt.title("Muestra de datos #1")
plt.scatter(xs, predictions, c='r')

print ("r^2:", lasso_model.score(X, ys))

In [ ]:
X = np.vander(xs2, 4)[:,:-1]
predictions = lasso_model.predict(X)

plt.scatter(xs2, ys2)
plt.title("Muestra de datos#2")
plt.scatter(xs2, predictions, c='r')

print ("r^2:", lasso_model.score(X, ys2))

In [ ]:
# Veamos los coeficientes de la regresión Lasso:    
lasso_model.coef_

### Elastic Net

In [ ]:
elastic_net = linear_model.ElasticNet(alpha=0.5, normalize=True)

# Ajustamos nuevamente, esta vez con regularizacion
X = np.vander(xs, 4)[:,:-1]
y = ys

elastic_net.fit(X, y)
predictions = elastic_net.predict(X)

plt.scatter(xs, ys)
plt.title("Muestra de datos #1")
plt.scatter(xs, predictions, c='r')

print ("r^2:", elastic_net.score(X, ys))

In [ ]:
X = np.vander(xs2, 4)[:,:-1]
predictions = elastic_net.predict(X)

plt.scatter(xs2, ys2)
plt.title("Muestra de datos#2")
plt.scatter(xs2, predictions, c='r')
print ("r^2:", elastic_net.score(X, ys2))

In [ ]:
# Veamos los coeficientes de la regresión ElasticNet:
elastic_net.coef_

### CV Optimizacion de lambda

In [ ]:
# instanciamos un modelo 
rlmcv = linear_model.RidgeCV(alphas=np.linspace(0.1,100, 1000), cv=3, normalize=True,scoring='r2')


# Ajustamos nuevamente nuestro modelo, esta vez con RidgeCV
X = np.vander(xs, 4)[:,:-1]
y = ys
rlmcv.fit(X, y)
predictions = rlmcv.predict(X)

plt.scatter(xs, ys)
plt.title("Muestra de datos #1")
plt.scatter(xs, predictions, c='r')
plt.show()
print ("r^2:", rlmcv.score(X, ys))
print ("alpha:", rlmcv.alpha_)

X = np.vander(xs2, 4)[:,:-1]
predictions = rlmcv.predict(X)

plt.scatter(xs2, ys2)
plt.title("Muestra de datos #2")
plt.scatter(xs2, predictions, c='r')
plt.show()
print ("r^2:", rlmcv.score(X, ys2))

In [ ]:
lassocv = linear_model.LassoCV(alphas=np.linspace(0.01,100, 1000), cv=3, normalize=True)
xs, ys = generate_data()


# Ajustamos nuevamente nuestro modelo, esta vez con LassoCV
X = np.vander(xs, 4)[:,:-1]
y = ys
lassocv.fit(X, y)
predictions = lassocv.predict(X)

plt.scatter(xs, ys)
plt.title("Muestra de datos #1")
plt.scatter(xs, predictions, c='r')
plt.show()
print ("r^2:", lassocv.score(X, ys))
print ("alpha:", lassocv.alpha_)

X = np.vander(xs2, 4)[:,:-1]
predictions = lassocv.predict(X)

plt.scatter(xs2, ys2)
plt.title("Muestra de datos #2")
plt.scatter(xs2, predictions, c='r')
plt.show()
print ("r^2:", lassocv.score(X, ys2))

In [ ]:
# Definimos el rango de de búsqueda del hiperparametro explicitamente
lm_lasso = linear_model.LassoCV(alphas=[0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01,\
                                        0.05, 0.1, 1, 5, 10],\
                                        normalize = False, cv = 5) 

model_cv = lm_lasso.fit(X_train, y_train)

model_cv.score(X_train, y_train)

model_cv.coef_
model_cv.intercept_
model_cv.alpha_
model_cv.score(X_train, y_train)

In [ ]:
best_alpha = model_cv.alpha_

#L1_wt : 0, the fit is ridge regression. 1, the fit is the lasso 

no_reg_model = sm.OLS(y_train, X_train_sm)

reg_model = no_reg_model.fit_regularized(alpha = best_alpha, L1_wt = 1)

reg_model.params
sns.scatterplot(x=reg_model.params, y=no_reg_model_params);

reg_residuals = y_train - reg_model.fittedvalues

linear_residuals = y_train - model.fittedvalues

sns.scatterplot(x = reg_residuals, y = linear_residuals)

In [ ]:
sm_prediction = reg_model.predict(X_test_sm)
sm_prediction

In [ ]:
skl_lasso = linear_model.Lasso(alpha = best_alpha, fit_intercept=True, normalize=False)

skl_lasso = skl_lasso.fit(X= X_train, y = y_train)

skl_prediction = skl_lasso.predict(X_test)

In [ ]:
skl_residuals = y_test - skl_prediction

sm_residuals = y_test - sm_prediction

sns.scatterplot(x = skl_residuals, y = sm_residuals)

In [ ]:

lasso_coef = np.insert(skl_lasso.coef_, 0, skl_lasso.intercept_)

sns.scatterplot(x = lasso_coef, y = reg_model.params);


In [ ]:
eval_measures.rmse(y_test, sm_prediction)
eval_measures.meanabs(y_test, sm_prediction)
metrics.r2_score(y_test, sm_prediction)
np.sqrt(metrics.mean_squared_error(y_test, skl_prediction))
metrics.mean_absolute_error(y_test, skl_prediction)
metrics.r2_score(y_test, skl_prediction)

### Ejemplo

In [ ]:
import random
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
  
random.seed(3)
boston = datasets.load_boston()
y = boston.target
X = pd.DataFrame(boston.data, columns=boston.feature_names)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
lr = linear_model.LinearRegression()
cv = KFold(5, shuffle=True)
scores = cross_val_score(lr, X_train, y_train, cv=cv, scoring='r2')
scores, scores.mean()

In [ ]:
model = linear_model.LassoCV().fit(X_train, y_train)
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum())

In [ ]:
model = linear_model.LassoCV(normalize=True).fit(X, y)
scores = cross_val_score(model, X, y, cv=cv, scoring='r2')
dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum())

In [ ]:
model = linear_model.RidgeCV(normalize=True).fit(X, y)
scores = cross_val_score(model, X, y, cv=cv, scoring='r2')
dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean())

### Ejemplo 2

In [ ]:
# Importamos las librerías

%matplotlib inline

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('../Data/salary.dat', delim_whitespace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Aplicar value_counts() a las series "sx", "dg", and "rk"

categories = ['sx', 'rk', 'dg']

for category in categories:
    print(df[category].value_counts())
    plt.bar(df[category].value_counts().index, df[category].value_counts().values, color='b',\
            alpha=0.5)
    plt.show()

In [ ]:
# Hacer unos violinplots a las series "sx", "dg", and "rk"

for category in categories:
    sns.violinplot(x=category, y='sl', data=df)
    plt.show()

# Repetir para "dg" y "rk"

In [ ]:
# Crear variables "dummy"

for category in categories:
    serie = df[category]
    dummies = pd.get_dummies(serie, drop_first= True, prefix=category)
    df = pd.concat([df, dummies], axis=1)

df.head()

In [ ]:
# Ajustamos el modelo usando solamente las variables cuantitativas y MCO

X = df[['yr', 'yd']]
y = df['sl']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=10)

lm = linear_model.LinearRegression()

model_1 = lm.fit(X_train, y_train)

print('Score model_1:', model_1.score(X_test, y_test))

In [ ]:
# Ajustamos el modelo usando solamente las variables cuantitativas aplicando regularización
#¿Hace falta normalizar los features antes aplicar regularización en este caso? ¿Qué unidades tienen los features?

lm_ridge = linear_model.RidgeCV(alphas=[0.1, 1, 10], normalize=True) 
# Definimos el rango de de búsqueda del hiperparametro explicitamente

model_2 = lm_ridge.fit(X_train, y_train)

print('Score model_2:', model_2.score(X_test, y_test))

# ¿Mejoraron los resultados?

In [ ]:
# Ahora entrenamos el modelo con todas las variables con MCO:

X_all = df.drop(['sx', 'rk', 'dg', 'sl'], axis=1) 

X_all_train, X_all_test, y_all_train, y_all_test = train_test_split(X_all, y, test_size=0.35, random_state=10)

model_3 = lm.fit(X_all_train, y_all_train)

print('Score model_3:', model_3.score(X_all_test, y_all_test))

In [ ]:
# Ahora entrenamos el modelo con todas las variables con Ridge:

lm_ridge = linear_model.RidgeCV(alphas=[0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01,\
                                        0.05, 0.1, 1, 5, 10],\
                                        normalize=True, cv=3) 
# Definimos el rango de de búsqueda del hiperparametro explicitamente

model_4 = lm_ridge.fit(X_all_train, y_all_train)

print('Score model_4:', model_4.score(X_all_test, y_all_test))

# ¿Mejoraron los resultados?

In [ ]:
lm_ridge.alpha_

In [ ]:
# Ahora entrenamos el modelo con todas las variables con Lasso:

lm_lasso = linear_model.LassoCV(alphas=[0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01,\
                                        0.05, 0.1, 1, 5, 10, 15, 25],\
                                        normalize=True, cv=3)

model_5 = lm_lasso.fit(X_all_train, y_all_train)

print('Score model_5:', model_5.score(X_all_test, y_all_test))

In [ ]:
lm_lasso.alpha_

# De articulo

In [ ]:
'''
LINEAR, RIDGE AND LASSO REGRESSION
'''
# importing requuired libraries
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge

# read test and train file
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print('\n\n---------DATA---------------\n\n')
print(train.head())

#splitting into training and test
## try building model with the different features and compare the result.
X = train.loc[:,['Outlet_Establishment_Year','Item_MRP']]
x_train, x_cv, y_train, y_cv = train_test_split(X,train.Item_Outlet_Sales,random_state=5)

print('--------Trainig Linear Regression Model---------------')
lreg = LinearRegression()
#training the model
lreg.fit(x_train,y_train)

#predicting on cv
pred = lreg.predict(x_cv)

#calculating mse
mse = np.mean((pred - y_cv)**2)
print('\nMean Sqaured Error = ',mse )

#Let us take a look at the coefficients of this linear regression model.
# calculating coefficients
coeff = DataFrame(x_train.columns)

coeff['Coefficient Estimate'] = Series(lreg.coef_)

print(coeff)

print('\n\nModel performance on Test data = ')
print(lreg.score(x_cv,y_cv))

print('\n\n---------Training Ridge Regression Model----------------')

ridge = Ridge()
ridge.fit(x_train,y_train)
pred1 = ridge.predict(x_cv)
mse_1 = np.mean((pred1-y_cv)**2)

print('\n\nMean Squared Error = ',mse_1)

# calculating coefficients
coeff = DataFrame(x_train.columns)
coeff['Coefficient Estimate'] = Series(ridge.coef_)
print(coeff)

print('\n\nModel performance on Test data = ')
print(ridge.score(x_cv,y_cv))


print('\n\n---------Training Lasso Regression Model----------------')

lasso = Lasso()
lasso.fit(x_train,y_train)
pred2 = lasso.predict(x_cv)
mse_2 = np.mean((pred2-y_cv)**2)

print('\n\nMean Squared Error = ',mse_2)

# calculating coefficients
coeff = DataFrame(x_train.columns)
coeff['Coefficient Estimate'] = Series(lasso.coef_)
print(coeff)

print('\n\nModel performance on Test data = ')
print(lasso.score(x_cv,y_cv))